In [2]:
import pandas as pd
from transformers import AutoTokenizer
import io

from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "/scratch/rohank__iitp/qwen2_5_7b_instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [6]:

prompt = "Who is Hrithik Roshan and some of his films?"
messages = [
    {"role": "system", "content": "Give correct answers."},
    {"role": "user", "content": prompt},
    {"role": "assistant", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=False
)

print(text)

<|im_start|>system
Give correct answers.<|im_end|>
<|im_start|>user
Who is Hrithik Roshan and some of his films?<|im_end|>
<|im_start|>assistant
Who is Hrithik Roshan and some of his films?<|im_end|>



In [8]:

df = pd.read_csv("/home/rohank__iitp/Work/niladri/fine_tune/conversation.csv")

# --- 2. Initialize the Tokenizer ---
# You can replace "mistralai/Mistral-7B-Instruct-v0.1" with your desired model from Hugging Face.
# This model is used here as an example because it has a well-defined chat template.
# tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

# --- 3. Process the data ---
SYSTEM_PROMPT = "You are an agent. Respond as an agent is answering."
processed_data = []
current_conversation_id = None
messages = []

# Iterate through each row of the DataFrame
for index, row in df.iterrows():
    conv_id = row['conversation_id']
    speaker = row['speaker']
    utterance = row['utterance']

    # If the conversation_id changes, reset the message list with the system prompt
    if conv_id != current_conversation_id:
        current_conversation_id = conv_id
        messages = [{"role": "system", "content": SYSTEM_PROMPT}]

    # Map the 'speaker' column to the appropriate role ('user' or 'assistant')
    role = "user" if speaker == "User" else "assistant"
    
    # Append the current message to the list
    messages.append({"role": role, "content": utterance})

    # Apply the chat template to the entire conversation history up to this point
    try:
        formatted_prompt = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=False
        )
    except Exception as e:
        print(f"Error applying template at row {index}: {e}")
        formatted_prompt = "Error"
    
    # Store the result by adding a new column to the row's data
    new_row_data = row.to_dict()
    new_row_data['chat_template'] = formatted_prompt
    processed_data.append(new_row_data)

# --- 4. Save the output to a new CSV file ---
output_df = pd.DataFrame(processed_data)
output_df.to_csv('/home/rohank__iitp/Work/niladri/fine_tune/tag_conversation.csv', index=False)

print("Processing complete. The output has been saved")
print("\nFirst 5 rows of the output file:")
print(output_df.head().to_string())

Processing complete. The output has been saved

First 5 rows of the output file:
   id  conversation_id  turn_no speaker                                                                                                                                                                                                                                                                                      utterance                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

## For new dataset by Sir

In [13]:
df = pd.read_csv("/home/rohank__iitp/Work/niladri/fine_tune/Final_mistral_dataset.csv")

# --- 2. Initialize the Tokenizer ---
# You can replace "mistralai/Mistral-7B-Instruct-v0.1" with your desired model from Hugging Face.

# --- 3. Process the data ---
SYSTEM_PROMPT = "You are an agent. Respond as an agent is answering."
processed_data = []
current_conversation_id = None
messages = []

# Iterate through each row of the DataFrame
for index, row in df.iterrows():
    conv_id = row['conversation_id']
    user_utterance = row['utterance']
    agent_reply = row['new_agent_reply']

    # If the conversation_id changes, reset the message list with the system prompt
    if conv_id != current_conversation_id:
        current_conversation_id = conv_id
        messages = [{"role": "system", "content": SYSTEM_PROMPT}]

    # In this new format, each row contains a user utterance and an agent reply.
    # We add both to the messages list for each row.
    
    # Add the user's message
    messages.append({"role": "user", "content": user_utterance})
    
    # Add the agent's reply
    messages.append({"role": "assistant", "content": agent_reply})

    # Apply the chat template to the entire conversation history up to this point
    try:
        formatted_prompt = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=False
        )
    except Exception as e:
        print(f"Error applying template at row {index}: {e}")
        formatted_prompt = "Error"
    
    # Store the result by adding a new column to the row's data
    new_row_data = row.to_dict()
    new_row_data['chat_template'] = formatted_prompt
    processed_data.append(new_row_data)

# --- 4. Save the output to a new CSV file ---
output_df = pd.DataFrame(processed_data)
output_df.to_csv('/home/rohank__iitp/Work/niladri/fine_tune/new_dataset_conversation.csv', index=False)

print("Processing complete. The output has been saved to 'formatted_conversations_new_format.csv'")
print("\nFirst 5 rows of the output file:")
print(output_df.head().to_string())

Processing complete. The output has been saved to 'formatted_conversations_new_format.csv'

First 5 rows of the output file:
   conversation_id  turn_no speaker                                                                                                      utterance                                                                                                                                                                                                                                                                                                                                                              new_agent_reply                                                                                                                                                                                                                                                                                                                                                                            